In [ ]:
!pip install cryptography
!pip install paramiko

In [ ]:
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.backends import default_backend

import paramiko
import os

In [ ]:
hostname = ''
port = 22
username = 'root'
password = ''

In [ ]:
directory = ''.join([x for x in hostname if x.isdigit()])[-5:]
                    
public_key_path = f'{directory}.pub'
private_key_path = f'{directory}'

In [ ]:
def generate_ssh_keys(private_key_path, public_key_path, passphrase=None):
    private_key = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048,
        backend=default_backend()
    )

    private_key_bytes = private_key.private_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PrivateFormat.TraditionalOpenSSL,
        encryption_algorithm=serialization.BestAvailableEncryption(passphrase.encode()) if passphrase else serialization.NoEncryption()
    )

    with open(private_key_path, "wb") as private_key_file:
        private_key_file.write(private_key_bytes)

    public_key = private_key.public_key()
    public_key_bytes = public_key.public_bytes(
        encoding=serialization.Encoding.OpenSSH,
        format=serialization.PublicFormat.OpenSSH
    )

    with open(public_key_path, "wb") as public_key_file:
        public_key_file.write(public_key_bytes)

In [ ]:
def load_key():
    try:
        remote_authorized_keys_path = "/root/.ssh/authorized_keys"
        ssh = paramiko.SSHClient()
        
        ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        ssh.connect(hostname, port, username, password)

        ssh.exec_command('mkdir -p /root/.ssh')
        ssh.exec_command('chmod 700 /root/.ssh')

        ssh.exec_command(f'sudo touch {remote_authorized_keys_path}')

        sftp = ssh.open_sftp()
        file_info = sftp.stat(remote_authorized_keys_path)
    
        if file_info.st_size == 0:
            print(f"Файл {remote_authorized_keys_path} пустой.")
            
            generate_ssh_keys(private_key_path, public_key_path)

            with open(public_key_path, "r") as pubkey_file:
                public_key = pubkey_file.read()
            
            ssh.exec_command(f'echo "{public_key}" > {remote_authorized_keys_path}')
            ssh.exec_command('chmod 600 /root/.ssh/authorized_keys')

            commands = [
            'sudo sed -i "s/^PasswordAuthentication yes/PasswordAuthentication no/" /etc/ssh/sshd_config',
            'sudo sed -i "s/^PermitRootLogin yes/PermitRootLogin prohibit-password/" /etc/ssh/sshd_config',
            'sudo sed -i "s/^PasswordAuthentication yes/PasswordAuthentication no/" /etc/ssh/sshd_config.d/50-cloud-init.conf',
            'sudo systemctl restart sshd',
            'sudo service ssh restart'
            ]

            for command in commands:
                _, stdout, _ = ssh.exec_command(command)
                print(stdout.read().decode())
    
        sftp.close()
    
        ssh.close()
    except Exception as e:
        pass

In [ ]:
load_key()

In [ ]:
private_key = paramiko.RSAKey.from_private_key_file(private_key_path)
ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh.connect(hostname, username=username, pkey=private_key)
_, stdout, _ = ssh.exec_command('hostname')
print(stdout.read().decode())

In [ ]:
_, stdout, _ = ssh.exec_command('sudo grep -i "PasswordAuthentication" /etc/ssh/sshd_config')
print(stdout.read().decode())